**废弃**

通过年报数据解决问题

财务报告年末日期

In [1]:
import pandas as pd
import numpy as np

from zipline.utils.numpy_utils import changed_locations

In [2]:
s1 = ['2016-12-31']*2 + ['2017-03-31']*2 + ['2017-06-30']*3 + ['2017-09-30']*2 + ['2017-12-31']*3 + ['2018-03-31']*3
s2 = ['2017-03-31']*2 + ['2017-06-30']*2 + ['2017-09-30']*3 + ['2017-12-31']*2 + ['2018-03-31']*3 + ['2018-06-30']*3
s3 = ['2016-12-31']*1 + ['2017-03-31']*3 + ['2017-06-30']*3 + ['2017-09-30']*2 + ['2017-12-31']*3 + ['2018-03-31']*3

In [3]:
dts1 = pd.to_datetime(s1)
dts2 = pd.to_datetime(s2)
dts3 = pd.to_datetime(s3)

In [4]:
df = pd.DataFrame([dts1,dts2,dts3]).T
dates = df.values

In [5]:
dates

array([['2016-12-31T00:00:00.000000000', '2017-03-31T00:00:00.000000000',
        '2016-12-31T00:00:00.000000000'],
       ['2016-12-31T00:00:00.000000000', '2017-03-31T00:00:00.000000000',
        '2017-03-31T00:00:00.000000000'],
       ['2017-03-31T00:00:00.000000000', '2017-06-30T00:00:00.000000000',
        '2017-03-31T00:00:00.000000000'],
       ['2017-03-31T00:00:00.000000000', '2017-06-30T00:00:00.000000000',
        '2017-03-31T00:00:00.000000000'],
       ['2017-06-30T00:00:00.000000000', '2017-09-30T00:00:00.000000000',
        '2017-06-30T00:00:00.000000000'],
       ['2017-06-30T00:00:00.000000000', '2017-09-30T00:00:00.000000000',
        '2017-06-30T00:00:00.000000000'],
       ['2017-06-30T00:00:00.000000000', '2017-09-30T00:00:00.000000000',
        '2017-06-30T00:00:00.000000000'],
       ['2017-09-30T00:00:00.000000000', '2017-12-31T00:00:00.000000000',
        '2017-09-30T00:00:00.000000000'],
       ['2017-09-30T00:00:00.000000000', '2017-12-31T00:00:00.000000000'

In [6]:
def _select_annual_indices(dates):
    """选取最近第n年年末财务报告位置辅助函数"""
    col_num = dates.shape[1]

    def get_annual_changed_locs(col_dates):
        is_year_end = pd.to_datetime(col_dates).is_year_end
        y_e_loc = np.argwhere(is_year_end).flatten()
        locs = changed_locations(np.diff(y_e_loc), True)
        print(y_e_loc[locs])
        return y_e_loc[locs]

    locs = []

    for col in range(col_num):
        singal_dates = dates[:,col]
        loc = get_annual_changed_locs(singal_dates)
        locs.append(loc)
        
    return locs

In [7]:
expect = np.array([[0,9],[7],[0,10]])

In [8]:
for e in expect:
    print(e)

[0, 9]
[7]
[0, 10]


In [9]:
_select_annual_indices(dates)

[0 1 9]
[7]
[0 9]


[array([0, 1, 9]), array([7]), array([0, 9])]

In [10]:
df

,0,1,2
0,2016-12-31,2017-03-31,2016-12-31
1,2016-12-31,2017-03-31,2017-03-31
2,2017-03-31,2017-06-30,2017-03-31
3,2017-03-31,2017-06-30,2017-03-31
4,2017-06-30,2017-09-30,2017-06-30
5,2017-06-30,2017-09-30,2017-06-30
6,2017-06-30,2017-09-30,2017-06-30
7,2017-09-30,2017-12-31,2017-09-30
8,2017-09-30,2017-12-31,2017-09-30
9,2017-12-31,2018-03-31,2017-12-31


In [11]:
is_year_end = df.applymap(lambda x:pd.to_datetime(x).is_year_end)

In [12]:
def _last(x):
    """给定位置序列，选取位置差为1的最后一个位置"""
    res = []
    num = len(x)
    selected = False
    for i in range(num-1, 0, -1):
        diff = x[i] - x[i-1]
        if x[i] - x[i-1] > 1:
            selected = False
        if (diff == 1) and (not selected):
            res.append(x[i])
            selected = True
    return list(reversed(res))       

In [13]:
res = []
for _, s in is_year_end.items():
    # 年末位置
    year_end_loc = np.argwhere(s).flatten()
    res.append(_last(year_end_loc))
print(res)

[[1, 11], [8], [11]]
